In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import os
import yadisk
import gdown
import json

import config
from parse_functions import *
from yandex_disk_func import *
from google_connector import *

# ссылка на основной файл гугл докс (на каждом лите отдельный клиент со ссылками на файлы с данными)
base_link = config.base_link
yandex_token = config.yandex_token
path = config.path
dashboadr_folder = config.dashboadr_folder

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = config.main_folder # путь к папке в формате /tmp
public_key = config.public_key # обычная ссылка на доступ к папке  из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [3]:
# забираем данные из основной управленческой таблицы
content = get_data_from_sheet(base_link)
sheet_names = pd.ExcelFile(content)

In [4]:
sheet_names.sheet_names

['Калининград МореАвто(ЮтоКарс)',
 'Екатеринбург АтлантикПРО',
 'АвтоБелогорье',
 'DoubleTint']

In [5]:
# если появился новый клиент
# мы проверяем, что этого названия НЕТ среди папок на Яндекс Диске
# созлаем папку Клиента
# внутри нее создаем папку для файлов дашбордов

folders_list = get_folder_name(yandex_folders['_embedded']['items'])
for name in sheet_names.sheet_names:
    if name not in folders_list:
        print(name)
        dashboard_folder = '/' + name + '/' + dashboadr_folder
        create_yandex_disk_folder(main_folder, name, yandex_token)
        create_yandex_disk_folder(main_folder, dashboard_folder, yandex_token)

Калининград МореАвто(ЮтоКарс)
Екатеринбург АтлантикПРО
АвтоБелогорье
DoubleTint


In [8]:
# обновляем список папок на Яндекс диске
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

# проходим через цикл по всем листам в управленческой таблице
# вызываем функцию и забираем названия папок на яндекс диске
folders_list = get_folder_name(yandex_folders['_embedded']['items'])

for name in sheet_names.sheet_names:
    print(name)
    
    cols_range = 'A:B' # задаем диапазон полей, которые нам нужны
    df = pd.read_excel(content, sheet_name=name, usecols=cols_range, names=['report_name', 'report_link'])
    # нормализуем назания отчетов - приводим в нижний регистр, обрезаем по краям, избаляемся от технических символов(перенос строки и тд)
    df['report_name'] = df['report_name'].str.lower().str.strip().str.replace('\n', ' ')
    # создаем отдельный список с названиями отчетов и проодим по нему через цикл
    reports_list = list(df['report_name'])
    # запускаем блок парсинга отчетов
    

    for report in reports_list:
        # print(report)
        if name=='Калининград МореАвто(ЮтоКарс)' and report == 'файлы дашбордов':
            parse_info = df[df['report_name']==report].reset_index(drop=True)
            base_link = parse_info['report_link'][0]
            print(base_link)
            # создаем папку для сохранения дашбордов, если ее нет
            # если она есть, то очищаем ее от файлов
            # current_directory = get_dashboards_folder(dashboadr_folder)
          
            # gdown.download_folder(base_link, output=current_directory, quiet=True, use_cookies=False)
            # получаем список файлов дашбордов
            current_files_list = list(os.listdir(current_directory))
            if len(current_files_list) > 0:
                for report in current_files_list:
                    get_files_to_yandex_disk(main_folder, base_link, name, report, yandex_token, flag='dashboard')
        # else:
        #     parse_info = df[df['report_name']==report].reset_index(drop=True)
        #     base_link = parse_info['report_link'][0]
            
        #     get_data_to_yandex_disk(main_folder, base_link, name, report, yandex_token, flag='xlsx')

200
Калининград МореАвто(ЮтоКарс)
https://drive.google.com/drive/folders/1A6RHtXKIMneWUkD-Fs2oLrfbVwDk07Er?usp=drive_link
Файл успешно загружен: Дашборд Калининград_АП.pbix
Файл успешно загружен: Дашборд Калининград_Выкуп.pbix
Файл успешно загружен: Дашборд Калининград_ОП.pbix
Файл успешно загружен: Дашборд Калининград_Парсинг.pbix
Файл успешно загружен: Посещение дашборда ЮтоКарс.pbix
Екатеринбург АтлантикПРО
АвтоБелогорье
DoubleTint


In [ ]:
def get_dashboards_folder(dashboadr_folder):
    # проверяем - существует папка для временного сохранения файлов или нет
    # если ее нет, то создаем
    # если папка существует, то проверяем в ней наличие файлов
    # если файлы в ней есть, то очищаем папку от содержимого
    # в самом конце выполняем загрузку файлов из гугл диска
    
    # получаем адрес текущей папки
    directory = os.getcwd()
    current_directory = os.path.join(directory, dashboadr_folder)
    if dashboadr_folder not in list(os.listdir(directory)):
        os.makedirs(current_directory) # создаем папку для сохранения файлов дашбордов
    else:
        current_files_list = list(os.listdir(os.path.join(directory, dashboadr_folder)))
        if len(current_files_list)!=0:
            for file in current_files_list:
                os.remove(os.path.join(current_directory, file))
    return current_directory

In [6]:
def get_files_to_yandex_disk(main_folder, base_link, name, report, yandex_token, flag='dashboard'):
    name = name + '/'
    if flag=='dashboard':
        dashboadr_folder_path = os.path.join(path, dashboadr_folder)
        content = os.path.join(dashboadr_folder_path, report)
        file_name = report
        name = name + dashboadr_folder
        file_path = name  +  dashboadr_folder + '/' + report

        delete_yandex_disk_file(main_folder, file_path, yandex_token)
        
        with open(content, 'rb') as f:
            try:
                upload_file_to_yandex_disk(main_folder, name, file_name, content, yandex_token)
            except:
                print('Ошибка при загрузке файла')
    else:
        spreadsheetId = get_sheet_id(base_link)
        content = download_google_data(spreadsheetId)
        file_name = report + '.xlsx'
        file_path = name + file_name
        
        delete_yandex_disk_file(main_folder, file_path, yandex_token)
        upload_file_to_yandex_disk(main_folder, name, file_name, content, yandex_token)

    # delete_yandex_disk_file(main_folder, file_path, yandex_token)
    # upload_file_to_yandex_disk(main_folder, name, file_name, content, yandex_token)

In [7]:
# функция для загрузки Excel файлов на Яндекс Диск
# на входе принимает путь к основной папке
# путь к папке сохранения
# название файла (без расширения)
# файл, который нужно сохранить
# токен Яндекс
def upload_file_to_yandex_disk(main_folder, file_path, file_name, content, yandex_token):
    if '/' not in file_path[0]:
        file_path = '/' + file_path
    # добавляем расширение файла к его названию
    # file_name = file_name + '.xlsx'
    # формируем итоговый путь сохраненияфайла
   
    file_path = file_path + '/' + file_name
    url_path = urlencode(dict(path=main_folder+file_path)) # кодируем полный путь сохранения файла вместе с его названием
    
    # добавляем флаг permanently=True для полного удаления файлов
    final_url = upload_url + url_path + '&overwrite=True'
    # формируем заголовки для дальнейших запросов
    headers = {
        'Content-Type': 'application/json', 
        'Accept': 'application/json', 
        'Authorization': f'OAuth {yandex_token}'
    }
   
    #вызываем метод get для получения ссылки для загрузки файла
    # это одноразовая ссылка, если загрузка не удалась, ссылку нужно получать заново
    res = requests.get(final_url, headers=headers) 
    # парсим ответ сервера
    data = json.loads(res.content)
    # забираем ссылку для загрузки файла
    download_link = data['href']
    # отправляем файл на яндекс диск
    status = requests.put(download_link, files={'file': content})
    if status.status_code == 201:
        print(f'Файл успешно загружен: {file_name}')
    else:
        print('Возможны ошибки - Проверьте Яндекс Диск')    

In [ ]:
dashboadr_folder

In [ ]:
from zipfile import ZipFile

In [ ]:


with ZipFile("metanit.zip", "w") as myzip:
    myzip.write("hello.txt")

In [ ]:
current_folder

In [10]:
current_folder = os.path.join(path, dashboadr_folder)

In [ ]:
files_list = list(os.listdir(current_folder))

In [ ]:
zip_folder_name = ('test.zip')
for file in files_list:
    with ZipFile(zip_folder_name, "w") as myzip:
        myzip.write(os.path.join(current_folder, file))

In [ ]:
files_list

In [11]:
import shutil
zip_name = 'dashboards_folder'
directory_name = current_folder
# Create 'path\to\zip_file.zip'
shutil.make_archive(zip_name, 'zip', directory_name)

'C:\\Users\\o.bogomolov\\Desktop\\Jupyter_notebook\\34_03_download_files_from_google_to_yandex\\dashboards_folder.zip'